# Restaurant Suggestions Project using OpenAI's GPT-3.5 Turbo

## Project Overview
The aim of this project is to develop a system that suggests restaurants to users based on their favorite food, restaurant, and city. It involves extracting relevant information from raw text data, building a structured DataFrame, and generating restaurant suggestions using OpenAI's GPT-3.5 Turbo API.

## Data Preparation
### 1. Reading and Processing Text Data
We read raw data from a text file, attempting UTF-8 decoding first and falling back to Latin-1 if it fails.

In [6]:
from openai import OpenAI
import pandas as pd
import json

In [7]:
try:
    with open("Food Data.txt", "r", encoding="utf-8") as file:
        raw_data = file.readlines()

except UnicodeDecodeError:
    # If 'utf-8' fails, fallback to 'latin-1'
    with open("Food Data.txt", "r", encoding="latin-1") as file:
        raw_data = file.readlines()

### 2. Extracting Information from Text Data
To extract relevant information (name, favorite food, favorite restaurant, city, and monthly spending), we use OpenAI's GPT-3.5 Turbo with a defined function schema. This allows us to extract structured information from unstructured text.

In [8]:
mykey = "your_key"
client = OpenAI(api_key=mykey)

In [9]:
food_extraction_function = [
    {
        "name": "extract_food_info",
        "description": "Extract name, favorite food, favorite restaurant, city, and monthly spending from text",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "Name of the student"},
                "favorite_food": {"type": "string", "description": "Favorite food"},
                "favorite_restaurant": {"type": "string", "description": "Favorite restaurant"},
                "city": {"type": "string", "description": "City of the student"},
                "monthly_spending": {"type": "integer", "description": "Monthly spending on eating out"},
            },
            "required": ["name", "favorite_food", "favorite_restaurant", "city", "monthly_spending"],
        },
    }
]

In [10]:
extracted_data = []
for line in raw_data:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": line}],
        functions=food_extraction_function,
        function_call={"name": "extract_food_info"},
    )

    response_data = json.loads(response.choices[0].message.function_call.arguments)
    extracted_data.append(response_data)

### 3. Creating a DataFrame
With the extracted information, we create a DataFrame to store the structured data.

In [11]:
df = pd.DataFrame(extracted_data)
df

,name,favorite_food,favorite_restaurant,city,monthly_spending
0,Ravi,Pizza and Burger,LaPinoz,Jaipur,9000
1,Sameer,Butter Chicken and Naan,Zaika,Delhi,5000
2,Kiran,Chole Bhature,Chandni Chowk,Delhi,2000
3,Maya,Rajma Chawal,Biryani Paradise,Bengaluru,3000
4,Akash,Thali meals,Saravana Bhavan,Chennai,4000
5,Neha,Aloo Gobi and Roti,Ghar Ki Rasoi,Mumbai,6000
6,Rohan,Biryani,Paradise Biryani,Hyderabad,4500
7,Aisha,Pav Bhaji,Aadyaa,Pune,3500
8,Dev,South Indian food,Sankalp,Chennai,2000
9,Mira,Masala Dosa,Iyengar Bakery,Bengaluru,2500


## Generating Suggestions
To generate restaurant suggestions, we create a function that suggests a restaurant based on the extracted information, using OpenAI's GPT-3.5 Turbo.

In [12]:
def generate_suggestions(row):
    # Extract the favorite food and restaurant from the DataFrame row
    favorite_food = row["favorite_food"]
    favorite_restaurant = row["favorite_restaurant"]
    city = row["city"]
    
    # Create the prompt for OpenAI's GPT-3.5-turbo
    prompt = f"Suggest just a name of restaurant or place along with the city to eat based on the favorite food: {favorite_food},favorite restaurant: {favorite_restaurant} and city:{city}."
    
    # Generate the suggestion using OpenAI API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=20,
    )
    
    # Return the suggestion from OpenAI
    return response.choices[0].message.content.strip()

In [13]:
df["Suggestion"] = df.apply(generate_suggestions, axis=1)
df

,name,favorite_food,favorite_restaurant,city,monthly_spending,Suggestion
0,Ravi,Pizza and Burger,LaPinoz,Jaipur,9000,The Burger Haven - Jaipur
1,Sameer,Butter Chicken and Naan,Zaika,Delhi,5000,Zaika Delights in Delhi
2,Kiran,Chole Bhature,Chandni Chowk,Delhi,2000,"""The Chole Bhature House"" in Delhi."
3,Maya,Rajma Chawal,Biryani Paradise,Bengaluru,3000,"Rajma Chawal Paradise, Bengaluru."
4,Akash,Thali meals,Saravana Bhavan,Chennai,4000,Madras Bhavan in Chennai.
5,Neha,Aloo Gobi and Roti,Ghar Ki Rasoi,Mumbai,6000,"""Flavors of Mumbai"" in Mumbai"
6,Rohan,Biryani,Paradise Biryani,Hyderabad,4500,Paradise Biryani House in Hyderabad
7,Aisha,Pav Bhaji,Aadyaa,Pune,3500,Pav Bhaji Junction in Pune
8,Dev,South Indian food,Sankalp,Chennai,2000,Savoury South - Chennai
9,Mira,Masala Dosa,Iyengar Bakery,Bengaluru,2500,Dosa Delight in Bengaluru


## Final Result
After processing, the DataFrame contains the extracted information and the suggested restaurants, ready for further analysis or user recommendations.

## Conclusion
This project showcases how to extract structured information from unstructured text data and use OpenAI's GPT-3.5 Turbo to generate contextual suggestions. It combines text processing, data extraction, and AI-based suggestions to create a user-centric restaurant recommendation system